# Graph Exploratory Data Analysis (EDA)
## Knowledge Graph / Wiki System

This notebook demonstrates **exploratory data analysis** of the Neo4j knowledge graph representing a company's internal documentation system.

### Objectives

- Analyze the graph structure and connectivity
- Explore relationships between Articles, Topics, Authors, and Tags
- Demonstrate graph traversal patterns for content discovery
- Provide insights into the knowledge base topology

### Prerequisites

- Neo4j database running via Docker Compose
- Database seeded with sample data (`make seed`)
- Python environment with `neo4j` driver and `python-dotenv`

---



In [41]:
# Import required libraries
import os
from neo4j import GraphDatabase, basic_auth
from dotenv import load_dotenv
import socket

# Load environment variables (same as the FastAPI app)
load_dotenv()

# Connection parameters from environment
# If running locally (outside Docker), use localhost; if in Docker, use neo4j
env_uri = os.getenv("NEO4J_URI")
if env_uri:
    NEO4J_URI = env_uri
else:
    # Try to detect if we're in Docker or running locally
    # Default to localhost for local execution
    NEO4J_URI = "bolt://localhost:7687"

# If URI contains 'neo4j:' hostname, try to resolve it
# If it fails, fallback to localhost (for local execution)
if "neo4j:" in NEO4J_URI and "localhost" not in NEO4J_URI:
    try:
        # Try to resolve the hostname
        socket.gethostbyname("neo4j")
        # If successful, we're in Docker network
        print(f"Detected Docker network, using: {NEO4J_URI}")
    except socket.gaierror:
        # Can't resolve neo4j, we're running locally
        NEO4J_URI = NEO4J_URI.replace("neo4j:", "localhost:")
        print(f"Running locally, using: {NEO4J_URI}")

NEO4J_USER = os.getenv("NEO4J_USER", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "password")

print(f"Connecting to Neo4j at {NEO4J_URI}...")

# Create driver and test connection
try:
    driver = GraphDatabase.driver(NEO4J_URI, auth=basic_auth(NEO4J_USER, NEO4J_PASSWORD))
    
    # Test connection
    with driver.session() as session:
        result = session.run("RETURN 1 AS test")
        test_value = result.single()["test"]
        print(f"✓ Connection successful! (test value: {test_value})")
    
    print("\nDriver initialized and ready for queries.")
except Exception as e:
    print(f"✗ Connection failed: {e}")
    print("\nTroubleshooting:")
    print("1. Make sure Neo4j is running: docker-compose up -d")
    print("2. If running locally, use: bolt://localhost:7687")
    print("3. If running in Docker, use: bolt://neo4j:7687")
    print("4. Check your .env file for NEO4J_URI setting")
    raise



Running locally, using: bolt://localhost:7687
Connecting to Neo4j at bolt://localhost:7687...
✓ Connection successful! (test value: 1)

Driver initialized and ready for queries.


## 1. Dataset Overview

Let's start by understanding the basic statistics of our knowledge graph: total nodes, node distribution by label, and relationship counts.



In [42]:
# Total number of nodes
with driver.session() as session:
    result = session.run("MATCH (n) RETURN count(n) AS total_nodes")
    total_nodes = result.single()["total_nodes"]
    print(f"Total nodes in the graph: {total_nodes}")



Total nodes in the graph: 15


In [43]:
# Number of nodes per label
with driver.session() as session:
    query = """
    MATCH (n)
    RETURN labels(n)[0] AS label, count(n) AS count
    ORDER BY count DESC
    """
    result = session.run(query)
    records = list(result)  # Convert to list to reuse
    
    print("Node distribution by label:")
    print("-" * 40)
    for record in records:
        print(f"{record['label']:20s}: {record['count']:3d} nodes")
    
    # Store for later use
    node_counts = {record['label']: record['count'] for record in records}



Node distribution by label:
----------------------------------------
Tag                 :   5 nodes
Topic               :   4 nodes
Author              :   3 nodes
Article             :   3 nodes


In [44]:
# Total number of relationships
with driver.session() as session:
    result = session.run("MATCH ()-[r]->() RETURN count(r) AS total_rels")
    total_rels = result.single()["total_rels"]
    print(f"Total relationships in the graph: {total_rels}")



Total relationships in the graph: 30


In [45]:
# Number of relationships per type
with driver.session() as session:
    query = """
    MATCH ()-[r]->()
    RETURN type(r) AS rel_type, count(r) AS count
    ORDER BY count DESC
    """
    result = session.run(query)
    records = list(result)  # Convert to list to reuse
    
    print("Relationship distribution by type:")
    print("-" * 50)
    for record in records:
        print(f"{record['rel_type']:25s}: {record['count']:3d} relationships")
    
    # Store for later use
    rel_counts = {record['rel_type']: record['count'] for record in records}



Relationship distribution by type:
--------------------------------------------------
HAS_TAG                  :   7 relationships
RELATED_TO_TOPIC         :   6 relationships
HAS_TOPIC                :   5 relationships
EXPERT_IN                :   4 relationships
WRITTEN_BY               :   4 relationships
RELATED_ARTICLE          :   4 relationships


**Interpretation:** These basic statistics provide a snapshot of the knowledge graph's scale and composition. Understanding the distribution of nodes and relationships helps identify which entity types are most prevalent and which relationships are most common. This foundation is essential for more complex analyses and helps validate that the graph structure matches expectations.

---

## 2. Structural Graph Exploration

Now we'll analyze the graph's connectivity patterns: node degrees, top-connected entities, and distribution patterns.



In [46]:
# Average degree per node type
with driver.session() as session:
    query = """
    MATCH (n)
    WITH labels(n)[0] AS label, n
    OPTIONAL MATCH (n)-[r1]->()
    OPTIONAL MATCH ()-[r2]->(n)
    WITH label, n, count(DISTINCT r1) + count(DISTINCT r2) AS degree
    RETURN label, 
           avg(degree) AS avg_degree,
           min(degree) AS min_degree,
           max(degree) AS max_degree
    ORDER BY avg_degree DESC
    """
    result = session.run(query)
    
    print("Average degree (connectivity) per node type:")
    print("-" * 60)
    print(f"{'Label':20s} {'Avg Degree':>12s} {'Min':>8s} {'Max':>8s}")
    print("-" * 60)
    for record in result:
        print(f"{record['label']:20s} {record['avg_degree']:12.2f} {record['min_degree']:8.0f} {record['max_degree']:8.0f}")



Average degree (connectivity) per node type:
------------------------------------------------------------
Label                  Avg Degree      Min      Max
------------------------------------------------------------
Article                      8.00        6       10
Topic                        5.25        4        7
Author                       2.67        2        4
Tag                          1.40        1        2


In [47]:
# Top Topics by degree (most connected topics)
with driver.session() as session:
    query = """
    MATCH (t:Topic)
    OPTIONAL MATCH (t)-[r1]->()
    OPTIONAL MATCH ()-[r2]->(t)
    WITH t, count(DISTINCT r1) + count(DISTINCT r2) AS degree
    RETURN t.name AS topic_name, 
           t.description AS description,
           degree
    ORDER BY degree DESC
    LIMIT 5
    """
    result = session.run(query)
    
    print("Top 5 most connected Topics:")
    print("-" * 70)
    for record in result:
        desc = record['description'] or 'N/A'
        print(f"Topic: {record['topic_name']}")
        print(f"  Description: {desc}")
        print(f"  Connections: {record['degree']}")
        print()



Top 5 most connected Topics:
----------------------------------------------------------------------
Topic: Artificial Intelligence
  Description: Study of intelligent agents and systems.
  Connections: 7

Topic: Knowledge Graphs
  Description: Graphs that store entities and their relationships.
  Connections: 5

Topic: Machine Learning
  Description: Algorithms that learn from data.
  Connections: 5

Topic: Natural Language Processing
  Description: Processing and understanding human language.
  Connections: 4



In [48]:
# Top Authors by degree (most connected authors)
with driver.session() as session:
    query = """
    MATCH (a:Author)
    OPTIONAL MATCH (a)-[r1]->()
    OPTIONAL MATCH ()-[r2]->(a)
    WITH a, count(DISTINCT r1) + count(DISTINCT r2) AS degree
    RETURN a.name AS author_name,
           a.affiliation AS affiliation,
           degree
    ORDER BY degree DESC
    LIMIT 5
    """
    result = session.run(query)
    
    print("Top 5 most connected Authors:")
    print("-" * 70)
    for record in result:
        affil = record['affiliation'] or 'N/A'
        print(f"Author: {record['author_name']}")
        print(f"  Affiliation: {affil}")
        print(f"  Connections: {record['degree']}")
        print()



Top 5 most connected Authors:
----------------------------------------------------------------------
Author: Alice Smith
  Affiliation: Research Lab X
  Connections: 4

Author: Bob Johnson
  Affiliation: University Y
  Connections: 2

Author: Carol Lee
  Affiliation: Data Science Team Z
  Connections: 2



In [49]:
# Distribution of Articles per Topic
with driver.session() as session:
    query = """
    MATCH (t:Topic)<-[:HAS_TOPIC]-(a:Article)
    WITH t.name AS topic_name, count(a) AS article_count
    RETURN topic_name, article_count
    ORDER BY article_count DESC
    """
    result = session.run(query)
    
    print("Distribution of Articles per Topic:")
    print("-" * 50)
    for record in result:
        print(f"{record['topic_name']:40s}: {record['article_count']:2d} articles")



Distribution of Articles per Topic:
--------------------------------------------------
Knowledge Graphs                        :  2 articles
Artificial Intelligence                 :  2 articles
Natural Language Processing             :  1 articles


In [50]:
# Distribution of Articles per Author
with driver.session() as session:
    query = """
    MATCH (a:Author)<-[:WRITTEN_BY]-(art:Article)
    WITH a.name AS author_name, count(art) AS article_count
    RETURN author_name, article_count
    ORDER BY article_count DESC
    """
    result = session.run(query)
    
    print("Distribution of Articles per Author:")
    print("-" * 50)
    for record in result:
        print(f"{record['author_name']:40s}: {record['article_count']:2d} articles")



Distribution of Articles per Author:
--------------------------------------------------
Alice Smith                             :  2 articles
Bob Johnson                             :  1 articles
Carol Lee                               :  1 articles


**Interpretation:** Structural analysis reveals the connectivity patterns in our knowledge graph. Topics with high degrees are central to the knowledge base and likely represent core domains. Authors with many connections are prolific contributors. The distribution of articles across topics and authors shows how knowledge is organized and who the key contributors are. This information is valuable for content recommendation systems and identifying knowledge gaps.

---

## 3. Graph Traversal Examples

Graph traversal is a key strength of graph databases. We'll demonstrate multi-hop queries that discover relationships and patterns not immediately visible.

### 3.1 Related Articles via Shared Topics or Tags

Find articles that share common topics or tags, enabling content discovery.



In [51]:
# Find articles related through shared topics
with driver.session() as session:
    query = """
    MATCH (a1:Article)-[:HAS_TOPIC]->(t:Topic)<-[:HAS_TOPIC]-(a2:Article)
    WHERE a1.id <> a2.id
    WITH a1, a2, collect(DISTINCT t.name) AS shared_topics
    RETURN a1.title AS article1,
           a2.title AS article2,
           shared_topics,
           size(shared_topics) AS shared_topic_count
    ORDER BY shared_topic_count DESC
    LIMIT 10
    """
    result = session.run(query)
    
    print("Articles related through shared Topics:")
    print("=" * 80)
    for record in result:
        print(f"\nArticle 1: {record['article1']}")
        print(f"Article 2: {record['article2']}")
        print(f"Shared Topics ({record['shared_topic_count']}): {', '.join(record['shared_topics'])}")
        print("-" * 80)



Articles related through shared Topics:

Article 1: Building a Company Knowledge Graph
Article 2: Using Graphs for Semantic Search
Shared Topics (1): Artificial Intelligence
--------------------------------------------------------------------------------

Article 1: Building a Company Knowledge Graph
Article 2: Introduction to Knowledge Graphs
Shared Topics (1): Knowledge Graphs
--------------------------------------------------------------------------------

Article 1: Introduction to Knowledge Graphs
Article 2: Building a Company Knowledge Graph
Shared Topics (1): Knowledge Graphs
--------------------------------------------------------------------------------

Article 1: Using Graphs for Semantic Search
Article 2: Building a Company Knowledge Graph
Shared Topics (1): Artificial Intelligence
--------------------------------------------------------------------------------


In [52]:
# Find articles related through shared tags
with driver.session() as session:
    query = """
    MATCH (a1:Article)-[:HAS_TAG]->(tag:Tag)<-[:HAS_TAG]-(a2:Article)
    WHERE a1.id <> a2.id
    WITH a1, a2, collect(DISTINCT tag.name) AS shared_tags
    RETURN a1.title AS article1,
           a2.title AS article2,
           shared_tags,
           size(shared_tags) AS shared_tag_count
    ORDER BY shared_tag_count DESC
    LIMIT 10
    """
    result = session.run(query)
    
    print("Articles related through shared Tags:")
    print("=" * 80)
    for record in result:
        print(f"\nArticle 1: {record['article1']}")
        print(f"Article 2: {record['article2']}")
        print(f"Shared Tags ({record['shared_tag_count']}): {', '.join(record['shared_tags'])}")
        print("-" * 80)



Articles related through shared Tags:

Article 1: Building a Company Knowledge Graph
Article 2: Introduction to Knowledge Graphs
Shared Tags (1): knowledge-graph
--------------------------------------------------------------------------------

Article 1: Building a Company Knowledge Graph
Article 2: Using Graphs for Semantic Search
Shared Tags (1): graph
--------------------------------------------------------------------------------

Article 1: Introduction to Knowledge Graphs
Article 2: Building a Company Knowledge Graph
Shared Tags (1): knowledge-graph
--------------------------------------------------------------------------------

Article 1: Using Graphs for Semantic Search
Article 2: Building a Company Knowledge Graph
Shared Tags (1): graph
--------------------------------------------------------------------------------


**Interpretation:** This traversal pattern is fundamental for recommendation systems. By finding articles that share topics or tags, we can suggest related content to users. This is more powerful than simple keyword matching because it leverages the graph structure to discover semantic relationships.

---

### 3.2 Articles within N Hops of a Given Topic

Explore the neighborhood of a topic to find all articles connected within a certain distance.



In [53]:
# Find all articles within 2 hops of "Knowledge Graphs" topic
topic_name = "Knowledge Graphs"

with driver.session() as session:
    query = """
    MATCH (t:Topic {name: $topic_name})
    MATCH path = (t)-[*1..2]-(art:Article)
    WHERE art IS NOT NULL
    WITH DISTINCT art, 
         min(length(path)) AS min_distance
    RETURN art.title AS article_title,
           art.summary AS summary,
           min_distance AS hops_from_topic
    ORDER BY min_distance, article_title
    """
    result = session.run(query, topic_name=topic_name)
    
    print(f"Articles within 2 hops of '{topic_name}':")
    print("=" * 80)
    for record in result:
        summary = record['summary'] or 'No summary'
        print(f"\nArticle: {record['article_title']}")
        print(f"Distance: {record['hops_from_topic']} hop(s)")
        print(f"Summary: {summary[:100]}...")
        print("-" * 80)



Articles within 2 hops of 'Knowledge Graphs':

Article: Building a Company Knowledge Graph
Distance: 1 hop(s)
Summary: How to design and deploy a knowledge graph for internal documentation....
--------------------------------------------------------------------------------

Article: Introduction to Knowledge Graphs
Distance: 1 hop(s)
Summary: Core concepts and use cases for knowledge graphs....
--------------------------------------------------------------------------------

Article: Using Graphs for Semantic Search
Distance: 2 hop(s)
Summary: Leverage graph structures to provide semantic search in an enterprise wiki....
--------------------------------------------------------------------------------


**Interpretation:** Variable-length path queries (`[*1..2]`) enable discovery of content that may not be directly linked but is semantically related through intermediate nodes. This supports exploratory search where users want to discover content around a topic of interest, even if it's not directly tagged with that topic.

---

### 3.3 Authors Connected via Shared Topics

Find authors who work on similar topics, enabling collaboration discovery.



In [54]:
# Find authors connected through shared topics (via articles or expertise)
with driver.session() as session:
    # First: Authors connected via articles sharing topics
    query1 = """
    MATCH (a1:Author)<-[:WRITTEN_BY]-(art:Article)-[:HAS_TOPIC]->(t:Topic)<-[:HAS_TOPIC]-(art2:Article)-[:WRITTEN_BY]->(a2:Author)
    WHERE a1.id <> a2.id
    RETURN a1.id AS author1_id, a1.name AS author1_name,
           a2.id AS author2_id, a2.name AS author2_name,
           collect(DISTINCT t.name) AS shared_topics
    """
    
    # Second: Authors connected via direct expertise
    query2 = """
    MATCH (a1:Author)-[:EXPERT_IN]->(t:Topic)<-[:EXPERT_IN]-(a2:Author)
    WHERE a1.id <> a2.id
    RETURN a1.id AS author1_id, a1.name AS author1_name,
           a2.id AS author2_id, a2.name AS author2_name,
           collect(DISTINCT t.name) AS shared_topics
    """
    
    # Combine results
    result1 = session.run(query1)
    result2 = session.run(query2)
    
    # Merge results by author pair
    author_pairs = {}
    
    for record in result1:
        key = tuple(sorted([record['author1_id'], record['author2_id']]))
        if key not in author_pairs:
            author_pairs[key] = {
                'author1': record['author1_name'],
                'author2': record['author2_name'],
                'topics': []
            }
        author_pairs[key]['topics'].extend(record['shared_topics'])
    
    for record in result2:
        key = tuple(sorted([record['author1_id'], record['author2_id']]))
        if key not in author_pairs:
            author_pairs[key] = {
                'author1': record['author1_name'],
                'author2': record['author2_name'],
                'topics': []
            }
        author_pairs[key]['topics'].extend(record['shared_topics'])
    
    # Remove duplicates and sort by number of shared topics
    results = []
    for key, data in author_pairs.items():
        unique_topics = list(set(data['topics']))
        if unique_topics:
            results.append({
                'author1': data['author1'],
                'author2': data['author2'],
                'topics': unique_topics,
                'count': len(unique_topics)
            })
    
    results.sort(key=lambda x: x['count'], reverse=True)
    
    print("Authors connected through shared Topics:")
    print("=" * 80)
    if results:
        for record in results[:10]:  # Limit to top 10
            print(f"\nAuthor 1: {record['author1']}")
            print(f"Author 2: {record['author2']}")
            print(f"Shared Topics ({record['count']}): {', '.join(record['topics'])}")
            print("-" * 80)
    else:
        print("No authors found connected through shared topics.")



Authors connected through shared Topics:

Author 1: Alice Smith
Author 2: Carol Lee
Shared Topics (1): Artificial Intelligence
--------------------------------------------------------------------------------

Author 1: Alice Smith
Author 2: Bob Johnson
Shared Topics (1): Knowledge Graphs
--------------------------------------------------------------------------------


**Interpretation:** This query demonstrates how graph traversal can identify potential collaborators or experts in related domains. By finding authors connected through shared topics (either through their articles or declared expertise), we can build social networks within the knowledge base and support team formation or expert finding.

---

### 3.4 Topic Neighborhoods (Depth = 2)

Explore the complete neighborhood around a topic, including related topics and all connected entities.



In [55]:
# Explore topic neighborhood with depth 2
topic_name = "Artificial Intelligence"

with driver.session() as session:
    query = """
    MATCH (start:Topic {name: $topic_name})
    OPTIONAL MATCH path1 = (start)-[:RELATED_TO_TOPIC*1..2]-(related:Topic)
    OPTIONAL MATCH path2 = (start)-[:RELATED_TO_TOPIC*1..2]-(t:Topic)<-[:HAS_TOPIC]-(art:Article)
    OPTIONAL MATCH path3 = (start)-[:RELATED_TO_TOPIC*1..2]-(t2:Topic)<-[:EXPERT_IN]-(auth:Author)
    
    WITH start, 
         collect(DISTINCT related) AS related_topics,
         collect(DISTINCT art) AS articles,
         collect(DISTINCT auth) AS authors
    
    RETURN start.name AS central_topic,
           start.description AS description,
           [t IN related_topics WHERE t IS NOT NULL | t.name] AS related_topic_names,
           size([t IN related_topics WHERE t IS NOT NULL]) AS related_topic_count,
           size([a IN articles WHERE a IS NOT NULL]) AS article_count,
           size([a IN authors WHERE a IS NOT NULL]) AS author_count
    """
    result = session.run(query, topic_name=topic_name)
    record = result.single()
    
    if record:
        print(f"Topic Neighborhood Analysis: {record['central_topic']}")
        print("=" * 80)
        print(f"Description: {record['description']}")
        print(f"\nRelated Topics (within 2 hops): {record['related_topic_count']}")
        for topic in record['related_topic_names']:
            print(f"  - {topic}")
        print(f"\nArticles connected: {record['article_count']}")
        print(f"Authors connected: {record['author_count']}")
    else:
        print(f"Topic '{topic_name}' not found.")



Topic Neighborhood Analysis: Artificial Intelligence
Description: Study of intelligent agents and systems.

Related Topics (within 2 hops): 4
  - Knowledge Graphs
  - Machine Learning
  - Natural Language Processing
  - Artificial Intelligence

Articles connected: 3
Authors connected: 3


**Interpretation:** Neighborhood analysis provides a comprehensive view of how a topic is embedded in the knowledge graph. This is useful for understanding topic centrality, discovering related content, and identifying subject matter experts. The multi-hop traversal reveals indirect connections that might not be obvious from direct relationships alone.

---

## 4. Visual Exploration (Optional)

Let's visualize a small subgraph to better understand the structure. We'll extract a sample and display it in a readable format.



In [56]:
# Extract a small subgraph for visualization
# Focus on one article and its immediate connections
with driver.session() as session:
    query = """
    MATCH (art:Article {id: 'article-1'})
    OPTIONAL MATCH (art)-[:HAS_TOPIC]->(t:Topic)
    OPTIONAL MATCH (art)-[:HAS_TAG]->(tag:Tag)
    OPTIONAL MATCH (art)-[:WRITTEN_BY]->(a:Author)
    OPTIONAL MATCH (art)-[:RELATED_ARTICLE]->(related:Article)
    
    RETURN art.title AS article_title,
           [t IN collect(DISTINCT t.name) WHERE t IS NOT NULL] AS topics,
           [tag IN collect(DISTINCT tag.name) WHERE tag IS NOT NULL] AS tags,
           [a IN collect(DISTINCT a.name) WHERE a IS NOT NULL] AS authors,
           [related IN collect(DISTINCT related.title) WHERE related IS NOT NULL] AS related_articles
    """
    result = session.run(query)
    record = result.single()
    
    if record:
        print("Subgraph: Article and its immediate connections")
        print("=" * 80)
        print(f"\n📄 Article: {record['article_title']}")
        print(f"\n📚 Topics ({len(record['topics'])}):")
        for topic in record['topics']:
            print(f"   • {topic}")
        print(f"\n🏷️  Tags ({len(record['tags'])}):")
        for tag in record['tags']:
            print(f"   • {tag}")
        print(f"\n✍️  Authors ({len(record['authors'])}):")
        for author in record['authors']:
            print(f"   • {author}")
        print(f"\n🔗 Related Articles ({len(record['related_articles'])}):")
        for related in record['related_articles']:
            print(f"   • {related}")
    else:
        print("Article 'article-1' not found.")



Subgraph: Article and its immediate connections

📄 Article: Building a Company Knowledge Graph

📚 Topics (2):
   • Artificial Intelligence
   • Knowledge Graphs

🏷️  Tags (3):
   • search
   • knowledge-graph
   • graph

✍️  Authors (1):
   • Alice Smith

🔗 Related Articles (2):
   • Introduction to Knowledge Graphs
   • Using Graphs for Semantic Search


In [57]:
# Visualize topic relationships as a simple text graph
with driver.session() as session:
    query = """
    MATCH (t1:Topic)-[r:RELATED_TO_TOPIC]->(t2:Topic)
    RETURN t1.name AS topic1, t2.name AS topic2
    ORDER BY t1.name, t2.name
    """
    result = session.run(query)
    
    print("Topic Relationship Graph:")
    print("=" * 80)
    print("(Direction: Topic1 → Topic2)\n")
    
    for record in result:
        print(f"  {record['topic1']:35s} → {record['topic2']}")



Topic Relationship Graph:
(Direction: Topic1 → Topic2)

  Artificial Intelligence             → Knowledge Graphs
  Artificial Intelligence             → Machine Learning
  Knowledge Graphs                    → Artificial Intelligence
  Machine Learning                    → Artificial Intelligence
  Machine Learning                    → Natural Language Processing
  Natural Language Processing         → Machine Learning


**Note:** For more advanced visualizations, you can use libraries like `networkx` and `matplotlib` to create graph plots. However, for large graphs, Neo4j Browser (http://localhost:7474) provides excellent interactive visualization capabilities.

---

## 5. Summary and Insights

### Key Findings

1. **Graph Structure**: The knowledge graph successfully models a documentation ecosystem with clear separation between content (Articles), concepts (Topics), people (Authors), and metadata (Tags).

2. **Connectivity Patterns**: 
   - Topics serve as central hubs connecting multiple articles
   - Authors contribute across different domains
   - Articles are richly connected through topics, tags, and explicit relationships

3. **Traversal Capabilities**: 
   - Multi-hop queries enable discovery of indirect relationships
   - Shared topics/tags provide natural content recommendation paths
   - Author networks reveal collaboration opportunities

4. **Use Cases Supported**:
   - **Semantic Search**: Find articles by traversing topic relationships
   - **Content Recommendation**: Discover related articles through shared concepts
   - **Expert Finding**: Identify authors by topic expertise
   - **Knowledge Discovery**: Explore topic neighborhoods to find related content

### Why Graph Databases Excel Here

Graph databases like Neo4j are ideal for knowledge management because:
- **Natural Modeling**: Relationships are first-class citizens, matching how knowledge is structured
- **Efficient Traversal**: Multi-hop queries are fast and intuitive
- **Flexible Schema**: Easy to add new relationship types as the knowledge base evolves
- **Discovery**: Graph patterns reveal insights that would require complex JOINs in relational databases

---

## Cleanup

Remember to close the driver connection when done.



In [58]:
# Close the driver connection
driver.close()
print("✓ Driver connection closed.")



✓ Driver connection closed.
